# Zadania

In [2]:
# Two-class MNIST 

from keras.datasets import mnist
import numpy as np

d1 = 5
d2 = 6

(mnist_x_train, mnist_y_train), (mnist_x_test, mnist_y_test) = mnist.load_data()

X_train = (mnist_x_train.astype('float32') / 255.).reshape((len(mnist_x_train), np.prod(mnist_x_train.shape[1:])))
y_train = mnist_y_train
X_test = (mnist_x_test.astype('float32') / 255.).reshape((len(mnist_x_test), np.prod(mnist_x_test.shape[1:])))
y_test = mnist_y_test

X_train = X_train[np.logical_or(y_train == d1, y_train == d2)]
y_train = y_train[np.logical_or(y_train == d1, y_train == d2)]
y_train[y_train==d1] = 0
y_train[y_train==d2] = 1
X_test = X_test[np.logical_or(y_test == d1, y_test == d2)]
y_test = y_test[np.logical_or(y_test == d1, y_test == d2)]
y_test[y_test==d1] = 0
y_test[y_test==d2] = 1

print X_train.shape
print X_test.shape


Using TensorFlow backend.


(11339, 784)
(1850, 784)


## Ćwiczenie 1 [5 pkt] ok?

Uzupełnij metody forward_pass oraz backward_pass w klasach ReLU, Sigmoid i Dense. Metoda forward_pass ma przyjmować batch inputów i zwracać batch outputów. Metoda backward_pass ma przyjmować batch inputów oraz batch pochodnych cząstkowych outputów i zwracać batch pochodnych cząstkowych inputów oraz wektor (**nie batch**) pochodnych cząstkowych wag. Jeśli wagi przechowujemy w macierzy dwuwymiarowej, to możemy najpierw policzyć pochodne cząstkowe w macierzy o takim samym kształcie, a następnie np. użyć .flat. 

Uwaga: dla warstw bez wag należy zwrócić None.

## Ćwiczenie 2 [4 pkt] ok ?

Uzupełnij metodę _forward_pass klasy Network. Metoda ta ma przyjmować batch inputów (X) i zwracać dwie rzeczy:
* inps - lista batchów inputów dla każdej warstwy w sieci (włącznie z X); te wartości będziemy używali w metodzie _backward_pass
* output - batch outputów z sieci (czyli $\mathbf{\hat y}$); output **nie** powinien być ostatnim elementem inps.

## Ćwiczenie 3 [5 pkt] ok ?

Uzupełnij metodę _backward_pass klasy Network. Zwróć uwagę, że pochodna funkcji kosztu po neuronach ostatniej warstwy jest już liczona w metodzie _fit_on_batch. Metoda ma zwracać listę layer_grads, której elementy to wektory pochodnych cząstkowych funkcji kosztu po kolejnych warstwach (zwrócone przez metodę Layer.backward_pass). Kolejność wektorów w tej liście ma być zgodna z kolejnością warstw w sieci.

## Ćwiczenie 4 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą i aktywacją Sigmoid na powyższych danych (dwuklasowy MNIST). Użyj MSE jako funkcji kosztu (oznacza to regresję do numeru klasy, co jest złym pomysłem, ale póki nie mamy klasy Crossentropy musi nam to wystarczyć). Użyj GD. Reportuj loss oraz accuracy.

## Ćwiczenie 5 [3 pkt]
Uzupełnić klasę Crossentropy, wzorując się na klasie MSE.

## Ćwiczenie  6 [3 pkt]
Uzupełnić klasę Momentum, wzorując się na klasie GD. Wzory można znaleźć tutaj: http://distill.pub/2017/momentum/

## Ćwiczenie 7 [3 pkt]
Naucz sieć neuronową z jedną warstwą ukrytą. Rozważ dwa przypadki: aktywację ReLU oraz Sigmoid. Czy jest sens używać ReLU jako ostatnią warstwę? Użyj Crossentropy jako funkcji kosztu. Użyj Momentum. Reportuj loss oraz accuracy.

## Ćwiczenie 8 [6 pkt]
Vanishing gradient.

Zadanie polega na zbadaniu zjawiska *vanishing gradient* w głębokich sieciach. Należy zmodyfikować kod warstwy Dense i dodać monitorowanie **normy euklidesowej** wektora delta_weights. Każdą warstwę Dense w trenowanej sieci należy monitorować oddzielnie. Po każdym wywołaniu metody fit_on_batch każdy z monitorów powinien zapamiętać nową normę. Po nauczeniu sieci dla każdej warstwy należy narysować wykres: poziomo - numer wywołania fit_on_batch, pionowo - norma delta_weights. Im niżej znajduje się warstwa Dense, tym silniej będzie zachodziło zjawisko *vanishing gradient*.

Naucz dwuwarstwową sieć z aktywacjami Sigmoid, reportując normy delta_weights. Powtórz to dla głębszej sieci (np. 6-10 warstw).

## Ćwiczenie 9 [4 pkt]
Przetestować kod z ćwiczenia 7. (dwuwarstwowa sieć) stosując inne inicjalizacje wag w warstwach Dense. Napisać własną inicjalizację wag, która sprawi, że sieć niczego się nie nauczy (init='stupid').


In [276]:
# Warstwy

class Layer():

    def forward_pass(self, input):
        # return output
        raise NotImplementedError()

    def backward_pass(self, input, output_grad):
        # return input_grad, weight_grad
        raise NotImplementedError()

    def update_weights(self, delta_weights):
        pass

    def debug_grad(self, evaluate_loss):
        return None

class ReLU(Layer):
    
    def forward_pass(self, input_):
        return np.maximum(np.zeros(input_.shape), input_)
        
    def backward_pass(self, input_, output_grad):
        # Ćwiczenie 1
        fbool = np.vectorize(lambda x: int(x>0))
        ones = fbool(input_)
#         print np.multiply(ones, output_grad), None
        return np.multiply(ones, output_grad), None
    
class Sigmoid(Layer):
    sigma = np.vectorize(lambda x: 1/float(1+np.exp(-x)))
    def forward_pass(self, input):
        # Ćwiczenie 1
#         return self.sigma(np.array(map(lambda f: [np.float128(x) for x in f], input)))
        return self.sigma(input)

    def backward_pass(self, input_, output_grad):
        # Ćwiczenie 1
        sigm_input = self.sigma(input_)
#         print reduce(np.multiply,
#             [output_grad,
#             sigm_input,
#             np.subtract(np.ones(sigm_input.shape), sigm_input)]), None
        return reduce(np.multiply,
            [output_grad,
            sigm_input,
            np.subtract(np.ones(sigm_input.shape), sigm_input)]), None

class Dense(Layer):

    def __init__(self, input_size, output_size, init = 'gaussian'):
        input_size += 1
        if init == 'zeros':
            self.weights = np.zeros((input_size, output_size))
        elif init == 'gaussian':
            self.weights = np.random.normal(
                0.,
                2. / (input_size + output_size),
                (input_size, output_size)
            )
        elif init == 'stupid':
            # Ćwiczenie 8
            raise NotImplementedError()            
        else:
            raise NotImplementedError()
        self.weights = np.asmatrix(self.weights)

    def forward_pass(self, input):
        # Ćwiczenie 1
        x = np.hstack((np.ones((input.shape[0], 1)), input))
        return x*self.weights
    
    
    def backward_pass(self, input_, output_grad): # TU MOZE BYC BLAD
        # Ćwiczenie 1
        input_ = np.hstack((np.ones((input_.shape[0], 1)), input_))
        
        dLdW = input_.T*output_grad
        dLdX = self.weights * output_grad.T
        
        input_grad = dLdX.T[:,1:]
        weight_grad = np.array(dLdW.flat)
#         print input_grad, weight_grad
        return input_grad, weight_grad

    def update_weights(self, delta_weights):
        # Ćwiczenie 7 - monitorowanie normy wektora delta_weights
#         print delta_weights.shape, self.weights.shape
        self.weights += delta_weights.reshape(self.weights.shape)

    def debug_grad(self, evaluate_loss):
        base = evaluate_loss()
        grad = []
        for (x, y), w in np.ndenumerate(self.weights):
            self.weights[x, y] = w + 0.0001
            changed = evaluate_loss()
            grad.append(10000. * (changed - base))
            self.weights[x, y] = w
        return grad

In [301]:
# Optimizers

class Optimizer():

    def calculate_deltas(self, grad):
        raise NotImplementedError()

class GD(Optimizer):

    def __init__(self, learning_rate):
        self.learning_rate = learning_rate

    def calculate_deltas(self, grad):
        return -self.learning_rate * grad

class Momentum(Optimizer):

    def __init__(self, alpha, beta):
        self.alpha = alpha
        self.beta = beta
        self.prev_z = 0
        
    def calculate_z(self, grad):
        z = self.beta*self.prev_z + grad
        self.prev_z = z
        return z
    
    def calculate_deltas(self, grad):
        return -self.alpha*self.calculate_z(grad)
        

In [339]:
# Funkcje kosztu

class Loss():

    def forward_pass(self, y, t):
        # return cost
        raise NotImplementedError()

    def backward_pass(self, y, t):
        # return y_grad
        raise NotImplementedError()

class MSE(Loss):

    def forward_pass(self, y, t):
        print y, t
        return np.average(0.5 * np.square(y - t))

    def backward_pass(self, y, t):
        return (y - t) / y.size

class Crossentropy(Loss):

    def forward_pass(self, y, t):
        print 'y:', y
        res = 0.0
        len_ = len(t)
        for i in range(len_):
            t_i = t[i]
            y_i = y[i]
            res += -t_i*np.log(y_iqq) - (1 - t_i)*np.log(1 - y_i)
        res /= len_
        print res
        return res
    def backward_pass(self, y, t):
        return ((np.divide(np.ones(t.shape) - t, np.ones(y.shape) - y)) - np.divide(t,y))/t.size
        


$$ L(\mathbf{y_{true}}, \mathbf{y_{pred}}) = L(\mathbf{t}, \mathbf{\hat y}) =\frac{1}{\mathrm{len}(\mathbf{t})} \sum_{j=1}^{\mathrm{len}(\mathbf{t})} - \mathbf{t}_j \log \mathbf{\hat y}_j - (1 - \mathbf{t}_j) \log (1 - \mathbf{\hat y}_j ) $$


$$\frac{\partial L}{\partial \mathbf{\hat y}} = \frac{1}{\mathrm{len}(\mathbf{t})}(-\frac{\mathbf{t}}{\mathbf{\hat y}} + \frac{1-\mathbf{t}}{1-\mathbf{\hat y}})$$


In [337]:
class Network():

    def __init__(self, loss, optimizer, metrics = []):
        self.layers = []
        self.loss = loss
        self.optimizer = optimizer
        self.metrics = metrics

    def add(self, layer):
        self.layers.append(layer)

    def fit(self, X, t, epochs, batch_size=256, print_stats=False):
        X = np.array(X)
        t = np.array(t)
        X = X.reshape(len(X), -1)
        t = t.reshape(len(t), -1)
        if X.shape[0] != t.shape[0]:
            raise ValueError("Array sizes don't match")

        for epoch in range(epochs):
            if print_stats:
                print("Epoch %d" % (epoch+1))
                print("    -> batch size: %d" % batch_size)
            rng_state = np.random.get_state()
            np.random.shuffle(X)
            np.random.set_state(rng_state)
            np.random.shuffle(t)
            pos = 0
            while pos < len(X):
                batch_X = X[pos:pos+batch_size]
                batch_t = t[pos:pos+batch_size]
                self._fit_on_batch(batch_X, batch_t)
                pos += batch_size
            if print_stats:
                _, y = self._forward_pass(X)
                print y,t
                l = self.loss.forward_pass(y, t)
                print l
                print("    -> loss: %f" % l)
                for m in self.metrics:
                    print("    -> %s: %f" % (m.__name__, m(y, t)))

    def predict(self, X):
        inps, out = self._forward_pass(X)
        return out

    def _fit_on_batch(self, batch_X, batch_t):
        inps, out = self._forward_pass(batch_X)
        layer_grads = self._backward_pass(
            inps,
            self.loss.backward_pass(out, batch_t)
        )
        grad = self._join(layer_grads)
        deltas = self.optimizer.calculate_deltas(grad)
        for l, d in zip(self.layers, self._split(deltas, layer_grads)):
            if not d is None:
                l.update_weights(d)

    def _join(self, grads):
        return np.concatenate([g for g in grads if not g is None])

    def _split(self, grads, layer_grads):
        out = []
        start = 0
        for l in layer_grads:
            if l is None:
                out.append(None)
            else:
                out.append(grads[start:start+len(l)])
                start += len(l)
        return out

    def _forward_pass(self, X):
        inps = []
        output = None

        # Ćwiczenie 2
        inps = [X]
        output = None
        for i in range(0,len(self.layers)):
            inps.append(self.layers[i].forward_pass(inps[i]))
        output = inps.pop()
        print output
        return inps, output

    def _backward_pass(self, inps, grad):
        # Ćwiczenie 3
        layer_grads = []
        length = len(inps)
        out_grad = grad
        for i in range(1, len(self.layers)+1):
            lr_ind = length - i
#             print layer_grads
#             print 'Layer %s: %s' % (lr_ind, self.layers[lr_ind].__class__)
#             print 'inp:', inps[lr_ind]
#             print 'grad:', layer_grads[0]
            k = self.layers[lr_ind].backward_pass(inps[lr_ind], out_grad)
#             print 'K', k
            res = k[1]
            out_grad = k[0]
            layer_grads = [res] + layer_grads
        return layer_grads

    def _debug_grads(self, X, t):
        layer_grads = []
        for l in self.layers:
            g = l.debug_grad(
                lambda: self.loss.forward_pass(self._forward_pass(X)[1], t)
            )
            if not g is None:
                g = np.array(np.array(g).flat)
            layer_grads.append(g)
        return layer_grads

In [340]:
##### Ćwiczenie 4

model = Network(MSE(), GD(0.1))
model.add(Dense(28*28, 28*28))
model.add(Sigmoid())
model.fit(X_train, y_train, 10, batch_size=X_train.shape[0], print_stats=True)

Epoch 1
    -> batch size: 11339
[[ 0.49977404  0.4981258   0.50551024 ...,  0.49715262  0.50001499
   0.49514273]
 [ 0.50218859  0.49679642  0.502968   ...,  0.49696085  0.50299734
   0.49623841]
 [ 0.50210716  0.49953333  0.50564114 ...,  0.49463684  0.49798808
   0.50125438]
 ..., 
 [ 0.49960296  0.49835554  0.50483702 ...,  0.49738332  0.49717017
   0.49502892]
 [ 0.50039311  0.49714135  0.504833   ...,  0.49578909  0.50100137
   0.48996793]
 [ 0.49967925  0.50166946  0.50041971 ...,  0.4983858   0.50059006
   0.49672656]]
[[ 0.49977205  0.49812429  0.50550715 ...,  0.4971514   0.50001331
   0.49514185]
 [ 0.50222366  0.49683221  0.5030019  ...,  0.49699678  0.50303273
   0.4962749 ]
 [ 0.5021007   0.49952722  0.50563383 ...,  0.49463099  0.49798192
   0.50124879]
 ..., 
 [ 0.49959186  0.49834495  0.50482454 ...,  0.4973731   0.49715946
   0.49501916]
 [ 0.50042638  0.49717528  0.50486514 ...,  0.49582313  0.50103487
   0.49000259]
 [ 0.49967798  0.50166836  0.50041796 ...,  0.4983

0.124952436326
    -> loss: 0.124952
Epoch 5
    -> batch size: 11339
[[ 0.49910466  0.49698961  0.50249892 ...,  0.49906033  0.49915766
   0.49365447]
 [ 0.50359893  0.49621564  0.50720326 ...,  0.49633719  0.50423156
   0.4952964 ]
 [ 0.50063349  0.50142059  0.5048145  ...,  0.49879363  0.50175538
   0.49921309]
 ..., 
 [ 0.50020059  0.49960714  0.49948488 ...,  0.50139802  0.5001875
   0.49577761]
 [ 0.49928757  0.49574045  0.50340919 ...,  0.49856928  0.50252313
   0.49339625]
 [ 0.50066813  0.5003726   0.49998127 ...,  0.49861328  0.49978534
   0.49616407]]
[[ 0.49911601  0.49700142  0.50250932 ...,  0.49907234  0.49916929
   0.49366697]
 [ 0.50365163  0.49626932  0.50725439 ...,  0.49639104  0.50428461
   0.49535098]
 [ 0.50063833  0.50142583  0.50481844 ...,  0.49879909  0.50176049
   0.49921886]
 ..., 
 [ 0.50020732  0.49961406  0.49949128 ...,  0.50140497  0.50019431
   0.49578477]
 [ 0.49934768  0.49580153  0.50346777 ...,  0.49863043  0.50258354
   0.49345819]
 [ 0.50068423 

0.124911808638
    -> loss: 0.124912
Epoch 9
    -> batch size: 11339
[[ 0.50412606  0.4970016   0.50327795 ...,  0.49676544  0.50416365
   0.4921359 ]
 [ 0.50252131  0.49602143  0.50389967 ...,  0.49725481  0.50185876
   0.49459778]
 [ 0.5020911   0.5000679   0.50428487 ...,  0.49845328  0.49958154
   0.49772015]
 ..., 
 [ 0.49823438  0.49748876  0.50288363 ...,  0.50127375  0.50022516
   0.50097903]
 [ 0.50071558  0.49936253  0.50831902 ...,  0.49379617  0.49513207
   0.49478008]
 [ 0.50420843  0.49985253  0.50165015 ...,  0.49859743  0.50419183
   0.49361998]]
[[ 0.50417632  0.49705276  0.5033269  ...,  0.49681673  0.50421424
   0.49218783]
 [ 0.50257102  0.49607203  0.50394806 ...,  0.49730553  0.50190883
   0.49464919]
 [ 0.502088    0.50006533  0.50428051 ...,  0.49845107  0.49957883
   0.49771835]
 ..., 
 [ 0.49824334  0.49749796  0.50289213 ...,  0.50128297  0.50023422
   0.50098855]
 [ 0.50070639  0.49935396  0.50830825 ...,  0.49378809  0.49512337
   0.49477245]
 [ 0.50425033

In [ ]:
# Ćwiczenie 9

In [ ]:
# Ćwiczenie 7
model = Network(Crossentropy(), GD(0.1))
model.add(Dense(28*28, 28*28))
model.add(Sigmoid())
model.fit(X_train, y_train, 10, batch_size=X_train.shape[0], print_stats=True)

In [241]:
np.random.seed(43)
d1 = Dense(input_size=4, output_size=3, init="gaussian")
d2 = Dense(input_size=4, output_size=3, init="zeros")
r = ReLU()
s = Sigmoid()
inp = np.random.random(4).reshape((1,-1)) - 0.5
out_grad_4 = np.random.random(4).reshape((1,-1)) - 0.5
out_grad_3 = np.random.random(3).reshape((1,-1)) - 0.5

print "inp:"
print inp.shape
print inp
print "out_grad_4:"
print out_grad_4.shape
print out_grad_4
print "out_grad_3:"
print out_grad_3.shape
print out_grad_3

print "Testing d1..."
print "d1.forward_pass(inp):"
t = d1.forward_pass(inp)
print t.shape
print t
print "d1.backward_pass(inp, out_grad_3):"
t = d1.backward_pass(inp, out_grad_3)
print t[0].shape
print t[0]
print t[1].shape
print t[1]

print "Testing d2..."
print "d2.forward_pass(inp):"
t = d2.forward_pass(inp)
print t.shape
print t
print "d2.backward_pass(inp, out_grad_3):"
t = d2.backward_pass(inp, out_grad_3)
print t[0].shape
print t[0]
print t[1].shape
print t[1]

print "Testing r..."
print "r.forward_pass(inp):"
t = r.forward_pass(inp)
print t.shape
print t
print "r.backward_pass(inp, out_grad_4):"
t = r.backward_pass(inp, out_grad_4)
print t[0].shape
print t[0]
print t[1]

print "Testing s..."
print "s.forward_pass(inp):"
t = s.forward_pass(inp)
print t.shape
print t
print "s.backward_pass(inp, out_grad_4):"
t = s.backward_pass(inp, out_grad_4)
print t[0].shape
print t[0]
print t[1]

inp:
(1, 4)
[[-0.05424164  0.16972465 -0.41749995  0.39709858]]
out_grad_4:
(1, 4)
[[-0.2019965  -0.23769518 -0.49487045  0.04320252]]
out_grad_3:
(1, 3)
[[-0.02440363  0.13637368  0.47820413]]
Testing d1...
d1.forward_pass(inp):
(1, 3)
[[-0.02707048 -0.20288487 -0.03403094]]
d1.backward_pass(inp, out_grad_3):
(1, 4)
[[-0.01685764  0.21647157  0.04530229  0.03971692]]
(15,)
[-0.02440363  0.13637368  0.47820413  0.00132369 -0.00739713 -0.02593858
 -0.0041419   0.02314597  0.08116303  0.01018852 -0.056936   -0.1996502
 -0.00969065  0.05415379  0.18989418]
Testing d2...
d2.forward_pass(inp):
(1, 3)
[[ 0.  0.  0.]]
d2.backward_pass(inp, out_grad_3):
(1, 4)
[[ 0.  0.  0.  0.]]
(15,)
[-0.02440363  0.13637368  0.47820413  0.00132369 -0.00739713 -0.02593858
 -0.0041419   0.02314597  0.08116303  0.01018852 -0.056936   -0.1996502
 -0.00969065  0.05415379  0.18989418]
Testing r...
r.forward_pass(inp):
(1, 4)
[[ 0.          0.16972465  0.          0.39709858]]
r.backward_pass(inp, out_grad_4):
(1,

    inp:
    (1, 4)
    [[-0.05424164  0.16972465 -0.41749995  0.39709858]]
    out_grad_4:
    (1, 4)
    [[-0.2019965  -0.23769518 -0.49487045  0.04320252]]
    out_grad_3:
    (1, 3)
    [[-0.02440363  0.13637368  0.47820413]]
    Testing d1...
    d1.forward_pass(inp):
    (1, 3)
    [[-0.02707048 -0.20288487 -0.03403094]]
    d1.backward_pass(inp, out_grad_3):
    (1, 4)
    [[-0.01685764  0.21647157  0.04530229  0.03971692]]
    (15,)
    [-0.02440363  0.13637368  0.47820413  0.00132369 -0.00739713 -0.02593858
     -0.0041419   0.02314597  0.08116303  0.01018852 -0.056936   -0.1996502
     -0.00969065  0.05415379  0.18989418]
    Testing d2...
    d2.forward_pass(inp):
    (1, 3)
    [[ 0.  0.  0.]]
    d2.backward_pass(inp, out_grad_3):
    (1, 4)
    [[ 0.  0.  0.  0.]]
    (15,)
    [-0.02440363  0.13637368  0.47820413  0.00132369 -0.00739713 -0.02593858
     -0.0041419   0.02314597  0.08116303  0.01018852 -0.056936   -0.1996502
     -0.00969065  0.05415379  0.18989418]
    Testing r...
    r.forward_pass(inp):
    (1, 4)
    [[ 0.          0.16972465  0.          0.39709858]]
    r.backward_pass(inp, out_grad_4):
    (1, 4)
    [[ 0.         -0.23769518  0.          0.04320252]]
    None
    Testing s...
    s.forward_pass(inp):
    (1, 4)
    [[ 0.48644291  0.5423296   0.39711514  0.59799036]]
    s.backward_pass(inp, out_grad_4):
    (1, 4)
    [[-0.050462   -0.05899789 -0.11847926  0.01038579]]
    None

## Ćwiczenie 2

In [242]:
from functools import partial
np.random.seed(43)
n = Network(loss=MSE(), optimizer=GD(learning_rate=0.001), metrics=[])
n.add(Dense(input_size=4, output_size=3, init="gaussian"))#0
n.add(ReLU())#1
n.add(Dense(input_size=3, output_size=4, init="gaussian"))#2
n.add(ReLU())#3
n.add(Dense(input_size=4, output_size=1, init="gaussian"))#4
n.add(Sigmoid())#5
inp = np.random.random((2,4)) - 0.5
inps, out = n._forward_pass(inp)
print "inp:"
print inp
for i, inp in enumerate(inps):
    print "inps[" + str(i) + "]:"
    print inp
print "out:"
print out

inp:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
inps[0]:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
inps[1]:
[[ 0.06231216 -0.25663962 -0.11549112]
 [ 0.03836232 -0.10585852 -0.03015728]]
inps[2]:
[[ 0.06231216  0.          0.        ]
 [ 0.03836232  0.          0.        ]]
inps[3]:
[[-0.04493685  0.4840028  -0.15711608 -0.22696511]
 [-0.0396103   0.48391873 -0.15615311 -0.24031931]]
inps[4]:
[[ 0.          0.4840028   0.          0.        ]
 [ 0.          0.48391873  0.          0.        ]]
inps[5]:
[[ 0.29948146]
 [ 0.29947621]]
out:
[[ 0.57431575]
 [ 0.57431447]]


    inp:
    [[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
     [ 0.17014516  0.20471561  0.10955987  0.04003446]]
    inps[0]:
    [[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
     [ 0.17014516  0.20471561  0.10955987  0.04003446]]
    inps[1]:
    [[ 0.06231216 -0.25663962 -0.11549112]
     [ 0.03836232 -0.10585852 -0.03015728]]
    inps[2]:
    [[ 0.06231216  0.          0.        ]
     [ 0.03836232  0.          0.        ]]
    inps[3]:
    [[-0.04493685  0.4840028  -0.15711608 -0.22696511]
     [-0.0396103   0.48391873 -0.15615311 -0.24031931]]
    inps[4]:
    [[ 0.          0.4840028   0.          0.        ]
     [ 0.          0.48391873  0.          0.        ]]
    inps[5]:
    [[ 0.29948146]
     [ 0.29947621]]
    out:
    [[ 0.57431575]
     [ 0.57431447]]

## Ćwiczenie 3

In [282]:
np.random.seed(43)
n = Network(loss=MSE(), optimizer=GD(learning_rate=0.001), metrics=[])
n.add(Dense(input_size=4, output_size=3, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=3, output_size=4, init="gaussian"))
n.add(ReLU())
n.add(Dense(input_size=4, output_size=1, init="gaussian"))
n.add(Sigmoid())

inp = np.random.random((2,4)) - 0.5
target = inp[:,0:1]
inps, out = n._forward_pass(inp)
grad = n.loss.backward_pass(out, target)
layer_grads = n._backward_pass(inps, grad)

print "inp:"
print inp
print "target:"
print target
for i, inp in enumerate(inps):
    print "inps[" + str(i) + "]:"
    print inp
print "out:"
print out
print "grad:"
print grad
for i, grad in enumerate(layer_grads):
    print "layer_grads[" + str(i) + "]:"
    print grad

inp:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
target:
[[ 0.06066317]
 [ 0.17014516]]
inps[0]:
[[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
 [ 0.17014516  0.20471561  0.10955987  0.04003446]]
inps[1]:
[[ 0.06231216 -0.25663962 -0.11549112]
 [ 0.03836232 -0.10585852 -0.03015728]]
inps[2]:
[[ 0.06231216  0.          0.        ]
 [ 0.03836232  0.          0.        ]]
inps[3]:
[[-0.04493685  0.4840028  -0.15711608 -0.22696511]
 [-0.0396103   0.48391873 -0.15615311 -0.24031931]]
inps[4]:
[[ 0.          0.4840028   0.          0.        ]
 [ 0.          0.48391873  0.          0.        ]]
inps[5]:
[[ 0.29948146]
 [ 0.29947621]]
out:
[[ 0.57431575]
 [ 0.57431447]]
grad:
[[ 0.25682629]
 [ 0.20208465]]
layer_grads[0]:
[  2.45711515e-05   0.00000000e+00   0.00000000e+00   2.67516826e-06
   0.00000000e+00   0.00000000e+00   9.94708194e-07   0.00000000e+00
   0.00000000e+00   4.30677425e-06   0.00000000e+00   0.00000000e+00
  -9.

    inp:
    [[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
     [ 0.17014516  0.20471561  0.10955987  0.04003446]]
    target:
    [[ 0.06066317]
     [ 0.17014516]]
    inps[0]:
    [[ 0.06066317 -0.08874451  0.22698799 -0.10080311]
     [ 0.17014516  0.20471561  0.10955987  0.04003446]]
    inps[1]:
    [[ 0.06231216 -0.25663962 -0.11549112]
     [ 0.03836232 -0.10585852 -0.03015728]]
    inps[2]:
    [[ 0.06231216  0.          0.        ]
     [ 0.03836232  0.          0.        ]]
    inps[3]:
    [[-0.04493685  0.4840028  -0.15711608 -0.22696511]
     [-0.0396103   0.48391873 -0.15615311 -0.24031931]]
    inps[4]:
    [[ 0.          0.4840028   0.          0.        ]
     [ 0.          0.48391873  0.          0.        ]]
    inps[5]:
    [[ 0.29948146]
     [ 0.29947621]]
    out:
    [[ 0.57431575]
     [ 0.57431447]]
    grad:
    [[ 0.25682629]
     [ 0.20208465]]
    layer_grads[0]:
    [  2.45711515e-05   0.00000000e+00   0.00000000e+00   2.67516826e-06
       0.00000000e+00   0.00000000e+00   9.94708194e-07   0.00000000e+00
       0.00000000e+00   4.30677425e-06   0.00000000e+00   0.00000000e+00
      -9.52974216e-07   0.00000000e+00   0.00000000e+00]
    layer_grads[1]:
    None
    layer_grads[2]:
    [ 0.          0.00700006  0.          0.          0.          0.00036236
      0.          0.          0.          0.          0.          0.          0.
      0.          0.          0.        ]
    layer_grads[3]:
    None
    layer_grads[4]:
    [ 0.11219329  0.          0.05429771  0.          0.        ]
    layer_grads[5]:
    None

## Ćwiczenie 5

In [ ]:
np.random.seed(43)
ce = Crossentropy()
y = np.random.random(10)
t = np.random.random(10)

print "y:"
print y
print "t:"
print t
print "ce.forward_pass(y,t):"
print ce.forward_pass(y,t)
print "ce.backward_pass(y,t):"
print ce.backward_pass(y,t)

    y:
    [ 0.11505457  0.60906654  0.13339096  0.24058962  0.32713906  0.85913749
      0.66609021  0.54116221  0.02901382  0.7337483 ]
    t:
    [ 0.39495002  0.80204712  0.25442113  0.05688494  0.86664864  0.221029
      0.40498945  0.31609647  0.0766627   0.84322469]
    ce.forward_pass(y,t):
    0.736415962327
    ce.backward_pass(y,t):
    [-0.27490047 -0.08104869 -0.10469935  0.10054647 -0.24509895  0.5272741
      0.11739401  0.0906406  -0.16913545 -0.05603779]

## Ćwiczenie 6

In [302]:
np.random.seed(43)
opt = Momentum(alpha=0.02, beta=0.99)
grad1 = np.random.random(5)
grad2 = np.random.random(5)
grad3 = np.random.random(5)
opt.calculate_deltas(grad1)
opt.calculate_deltas(grad2)
opt.calculate_deltas(grad3)

print "grad1:"
print grad1
print "grad2:"
print grad2
print "grad3:"
print grad3

print "opt.calculate_deltas(grad1):"
print opt.calculate_deltas(grad1)
print "opt.calculate_deltas(grad2):"
print opt.calculate_deltas(grad2)
print "opt.calculate_deltas(grad3):"
print opt.calculate_deltas(grad3)

grad1:
[ 0.11505457  0.60906654  0.13339096  0.24058962  0.32713906]
grad2:
[ 0.85913749  0.66609021  0.54116221  0.02901382  0.7337483 ]
grad3:
[ 0.39495002  0.80204712  0.25442113  0.05688494  0.86664864]
opt.calculate_deltas(grad1):
[-0.02919466 -0.0529381  -0.0209018  -0.01117572 -0.04443381]
opt.calculate_deltas(grad2):
[-0.04608546 -0.06573052 -0.03151603 -0.01164424 -0.05866444]
opt.calculate_deltas(grad3):
[-0.05352361 -0.08111416 -0.03628929 -0.0126655  -0.07541077]


    grad1:
    [ 0.11505457  0.60906654  0.13339096  0.24058962  0.32713906]
    grad2:
    [ 0.85913749  0.66609021  0.54116221  0.02901382  0.7337483 ]
    grad3:
    [ 0.39495002  0.80204712  0.25442113  0.05688494  0.86664864]
    opt.calculate_deltas(grad1):
    [-0.02919466 -0.0529381  -0.0209018  -0.01117572 -0.04443381]
    opt.calculate_deltas(grad2):
    [-0.04608546 -0.06573052 -0.03151603 -0.01164424 -0.05866444]
    opt.calculate_deltas(grad3):
    [-0.05352361 -0.08111416 -0.03628929 -0.0126655  -0.07541077]